Misspellings of curse words?  

When building vocab, consider punctuation. Tokenize it first? Right now it just splits on whitespace. Moses probably has something for this

In [40]:
from time import time


import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
np.random.seed(1234) # help reproducibility
olidData = 'data/OLIDv1.0/olid-training-v1.0.tsv'

In [3]:
olid = open(olidData)
df = pd.read_csv(olid, sep='\t')
olid.close()
df = df.drop(df.columns[[0,3,4]], 1)


In [4]:
data_top = df.head()
print(data_top)

                                               tweet subtask_a
0  @USER She should ask a few native Americans wh...       OFF
1  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF
2  Amazon is investigating Chinese employees who ...       NOT
3  @USER Someone should'veTaken" this piece of sh...       OFF
4  @USER @USER Obama wanted liberals &amp; illega...       NOT


In [5]:
text = np.array(df['tweet'])
offen = np.array(df['subtask_a'])

from sklearn.model_selection import train_test_split
text_train, text_test, offen_train, offen_test = train_test_split(text, offen, test_size=0.2, random_state=4)


In [15]:
from sklearn.pipeline import Pipeline,FeatureUnion
pipe = Pipeline([
    ('feats', FeatureUnion([
    ('tfidf', TfidfVectorizer()),
    ('cvec', CountVectorizer()),
    ])),
    
    ('lr' ,LogisticRegression())
])

In [17]:
pipe_parms = [{
    'feats__tfidf__max_features' : [100,500],
    'feats__tfidf__ngram_range' : [(1,1),(1,2)],
    'feats__cvec__max_features' : [100,500],
    'feats__cvec__ngram_range' : [(1,1),(1,2)],
}]

In [25]:
labels = pipe.predict(text_test)

In [28]:
get_metrics(offen_test, labels)

Accuracy: 0.7553
Precision: 0.7468
Recall: 0.7553
F1 Score: 0.746


In [18]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(pipe, param_grid= pipe_parms, cv=5)

In [39]:
train_predict_display_model(classifier=pipe)

/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.7553
Precision: 0.7468
Recall: 0.7553
F1 Score: 0.746


In [37]:
####Evaluation metrics


from sklearn import metrics
import numpy as np
import pandas as pd

def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        
def train_predict_display_model(classifier, 
                        train_features=text_train, 
                        test_features=text_test, 
                        train_labels=offen_train, 
                        test_labels=offen_test):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    get_metrics(test_labels,predictions)   

    
